## **CIS 520: Machine Learning**
## **Array Practice**


- **Content Creators:** Pengrui Wang, Siyun Hu, Ani Cowlagi
- **Content Reviewers:** Lyle Ungar
- **Objectives:** This is a self-practice for working with numpy arrays.

If you are not familar with NumPy, feel free to refer to [NumPy Tutorial](https://github.com/ageron/handson-ml/blob/master/tools_numpy.ipynb) and [Vectorization Notebook](https://drive.google.com/file/d/19IJK8mxCg8ww9W0lJQuAaVbzek4wBLsw/view?usp=sharing).


In [ ]:
#@markdown Tell us your thoughts about what you want to learn.
thoughts = '' #@param {type:"string"}
import time
try: t0;
except NameError: t0=time.time()

## **Autograding and the PennGrader**

First, you'll need to set up the PennGrader, which we'll be using throughout the semester to help you with your homeworks and worksheeets.

PennGrader is not only **awesome**, but it was built by an equally awesome person: Leo Murri.  Today, Leo works as a data scientist at Amazon!

PennGrader was developed to provide students with *instant* feedback on their answer. You can submit your answer and know whether it's right or wrong instantly. We then record your most recent answer in our backend database.

Enter your PennID (numbers not letters!) in the specified section.

### Imports and Setup (Do Not Modify This Section)

In [ ]:
%%capture
!pip install penngrader


In [ ]:
import random 
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from numpy.linalg import *

import dill
import base64

In [ ]:
# For autograder only, do not modify this cell. 
# True for Google Colab, False for autograder
NOTEBOOK = (os.getenv('IS_AUTOGRADER') is None)
if NOTEBOOK:
    print("[INFO, OK] Google Colab.")
else:
    print("[INFO, OK] Autograder.")
    sys.exit()

[INFO, OK] Google Colab.


### Insert PennID here!

In [ ]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO 
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER#

In [ ]:
import penngrader.grader

grader = penngrader.grader.PennGrader(homework_id = 'CIS_5200_202230_HW_Vectorization_WS', student_id = STUDENT_ID)

PennGrader initialized with Student ID: 99999999

Make sure this correct or we will not be able to store your grade


In [ ]:
# A helper function for grading utils
def grader_serialize(obj):        # A helper function
    '''Dill serializes Python object into a UTF-8 string'''
    byte_serialized = dill.dumps(obj, recurse = True)
    return base64.b64encode(byte_serialized).decode("utf-8")

## **Introduction**

Python for loops are inherently far slower than their C counterpart as these loops are interpreted at runtime rather than at compile time. As a result, they should be avoided at all costs whenever speed is important. Numpy tries to solve this issue. 

NumPy is a scientific computing library for Python. It allows users to work efficiently with large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays. NumPy is used extensively by almost every core scientific computing package used in Python (Scikit-Learn, Scipy, Pandas, Matplotlib, etc.). As a result, it is a key goal of this course to become proficient at working with NumPy arrays -- no more loops! 

This worksheet walks you through how NumPy can be used to *vectorize* array operations. You will benefit a lot if you keep the idea of vectorization in mind when you write Python code. 




### Comparison Metrics

Jupyter notebooks provide built-in commands called **magics** to address common problems in the data analysis workflow. For example,

- *line* magics are denoted by a single `%`
- *cell* magics that apply to the entire cell are denoted by a double `%%`


Perhaps, the most useful magic functions are `time` and `timeit`

- `time` times the amount of time used for a single run of the line of code or cell
- `timeit` runs multiple trials of the line of code or cell to provide a more accurate measurement of the execution time

In [ ]:
%time odd_squares = [num ** 2 for num in range(100000) if num % 2 == 1]

CPU times: user 24.7 ms, sys: 1.03 ms, total: 25.8 ms
Wall time: 27.5 ms


In [ ]:
%timeit odd_squares = [num ** 2 for num in range(100000) if num % 2 == 1]

21.6 ms ± 618 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# we can also time entire cells
%%time

nums = range(100000)
odd_squares = []

for num in nums:
  if num % 2 == 1:
    odd_squares.append(num**2)

CPU times: user 29.1 ms, sys: 0 ns, total: 29.1 ms
Wall time: 29.2 ms


In [ ]:
%%timeit

nums = range(100000)
odd_squares = []

for num in nums:
  if num % 2 == 1:
    odd_squares.append(num**2)

24.2 ms ± 705 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Interpreting timeit output**

- x loops: the number of loops (factors of 10) of the code needed to exceed 0.2 seconds of wall time
- best of 3: run the loop trials three times, take the best results
- x ms per loop: the amount of time the code took to execute under the best run

By leveraging `time` and `timeit`, we could evaluate the time we use to run different cells, so as to understand the advantages of vectorization.

### Imports

- we need to import the NumPy module below in order to use it
- we often use the `as` syntax to alias the module name to an abbreviation

In [ ]:
# subsequent references to the numpy module can use 'np'
import numpy as np
import math

## Vectorization

For example, let's say we want to generate a 768x1024 array based on the formula $sin(xy/40.5)$. A bad option would be to do the math in python using nested loops:

In [ ]:
%%timeit
# for loop - inefficient
data = np.empty((768, 1024))
for y in range(768):
    for x in range(1024):
        data[y, x] = math.sin(x*y/40.5)  

346 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
x_coords = np.arange(0, 1024)  # [0, 1, 2, ..., 1023]
y_coords = np.arange(0, 768)   # [0, 1, 2, ..., 767]
X, Y = np.meshgrid(x_coords, y_coords)
data = np.sin(X*Y/40.5)

26.6 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Rather than for loop, we use `np.meshgrid` and `np.sin` to vectorize this algorithm. The result shows that, vectorization reduces the execution time by more than 10 times.

Here is another example that uses `np.sum` to vectorize a summation operation.

In [ ]:
%%timeit
# Don't do this!!
vec = np.array(range(10000))
sum_v = 0
for i in range(10000):
  sum_v += vec[i]

2.64 ms ± 65.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
# Vectorize instead
vec = np.array(range(10000))
sum_v = np.sum(vec)

881 µs ± 19.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


However, what is the behavior of  `np.sum()` if our array has more than one dimension?

- Use the `axis` argument: specifies which axis to sum along
- Many other vectorized functions take the axis argument, so keep in mind which dimension you want the operation applied to

In [ ]:
A = np.array([[1,2,3], [4,5,6], [7,8,9]])
print(A)
print(A.shape)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
(3, 3)


In [ ]:
# By default, axis=None and np.sum will sum all elements of the array
print(np.sum(A))

45


In [ ]:
# axis=0 sums along rows, producing column totals
print(np.sum(A, axis=0))

[12 15 18]


In [ ]:
# axis=0 sums along columns, producing row totals
print(np.sum(A, axis=1))

[ 6 15 24]


## Case Study: YamSlam!
![](https://alliance.seas.upenn.edu/~cis520/dynamic/2019/wiki/images/yamslam.png)

Below is a comprehensive case that uses NumPy array.

- Given 3 chances to roll,
how likely is it that you
will roll 5 of a kind?
- Strategy: Pick the most
common #, and re-roll
dice that don’t match

### Rolling 5 Dice



```python
y = np.zeros(5)       
roll_idx = np.array(range(5))
y[roll_idx] = np.floor(np.random.uniform(0,6, roll_idx.shape))
```

---


- `np.zeros(5)` creates an array of zeros of shape (5,)
  - **Note:** this is a 1D array with 5 elements, which is different than an 2D array with only one column, shape (5,1)
- `np.array(range(5))` creates an array \[0,1,2,3,4\] of shape (5,)
- `np.random.uniform(0,6, roll_idx.shape)` samples from the uniform distribution in the range \[0, 6) in the given shape
- `np.floor` rounds down to the nearest integer
  - **Note:** we're using zero-indexed dice, so our possible rolls are \[0,1,2,3,4,5\]
  
*Tip: If you are unsure of a function's arguments or return signature, you can run `help(function_name)` to print the docstring*

In [ ]:
help(np.random.uniform)

Help on built-in function uniform:

uniform(...) method of numpy.random.mtrand.RandomState instance
    uniform(low=0.0, high=1.0, size=None)
    
    Draw samples from a uniform distribution.
    
    Samples are uniformly distributed over the half-open interval
    ``[low, high)`` (includes low, but excludes high).  In other words,
    any value within the given interval is equally likely to be drawn
    by `uniform`.
    
    .. note::
        New code should use the ``uniform`` method of a ``default_rng()``
        instance instead; please see the :ref:`random-quick-start`.
    
    Parameters
    ----------
    low : float or array_like of floats, optional
        Lower boundary of the output interval.  All values generated will be
        greater than or equal to low.  The default value is 0.
    high : float or array_like of floats
        Upper boundary of the output interval.  All values generated will be
        less than or equal to high.  The default value is 1.0.
    size 

#### What is the value and shape of y?

In [ ]:
y = np.zeros(5)       
roll_idx = np.array(range(5))
y[roll_idx] = np.floor(np.random.uniform(0,6, roll_idx.shape))
print(y)
print(y.shape)

[0. 4. 3. 5. 0.]
(5,)


In [ ]:
y = np.zeros((5,1))       
roll_idx = np.array(range(5))
# note the need for a second index, since y is now a 2D array!
y[roll_idx, 0] = np.floor(np.random.uniform(0,6, roll_idx.shape))
print(y)
print(y.shape)

[[2.]
 [2.]
 [1.]
 [1.]
 [5.]]
(5, 1)


In [ ]:
y = np.zeros((5,5))       
roll_idx = np.array(range(5))
# : is shorthand for selecting all indices along an axis. More on indexing later
# What happens if we run y[:, roll_idx] instead?
y[roll_idx, :] = np.floor(np.random.uniform(0,6, roll_idx.shape))
print(y)
print(y.shape)

[[4. 4. 5. 2. 3.]
 [4. 4. 5. 2. 3.]
 [4. 4. 5. 2. 3.]
 [4. 4. 5. 2. 3.]
 [4. 4. 5. 2. 3.]]
(5, 5)


### Re-rolling

#### Random number generation
```python
np.random.seed(0)
```

---

- `np.random.seed(0)` sets the seed of the Numpy's random number generator to 0, ensuring that subsequent calls to `np.random` functions are reproducible
  - note that any `int`, not just 0, will work as a reproducible seed

In [ ]:
# Here we're using the random seed to make sure our first roll is reproducible
np.random.seed(0)
y = np.zeros(5)       
roll_idx = np.array(range(5))
y[roll_idx] = np.floor(np.random.uniform(0,6, roll_idx.shape))
print(y)

[3. 4. 3. 3. 2.]


- 3 is the most common, so we want to re-roll indices 1 and 4

#### How do we get the indices to re-roll?

In [ ]:
# check which entries of y aren't 3 
y != 3

array([False,  True, False, False,  True])

In [ ]:
# return the indices that are "True" -- interpreted as non-zero by Numpy
# note the return shape! np.nonzero returns a tuple of arrays, one for each axis
np.nonzero(y != 3)

(array([1, 4]),)

In [ ]:
# assign the new roll_idx
roll_idx = np.nonzero(y != 3)[0]
print(roll_idx)

[1 4]


#### The general re-roll case


In [ ]:
# count the number of dice we've rolled for each number
counts = [sum(y == i) for i in range(6)]
print(counts)

[0, 0, 1, 3, 1, 0]


In [ ]:
# Find the idx of the most common roll
max_idx = np.argmax(counts)
print(max_idx)

3


In [ ]:
# update the indices to re-roll accordingly
roll_idx = np.nonzero(y != max_idx)[0]    
print(roll_idx)

[1 4]


### Putting it all together

In [ ]:
def yamslam():
  """Plays one round of yamslam, re-rolling 5 dice up to 3 times.
  Also prints exuberantly if we do get a yamslam.
  Returns:
    int: 1 if we got a yamslam, 0 if not
  """
  y = np.zeros(5)       # for the 5 dice, to store the values
  roll_idx = np.array(range(5)) # allows us to set all 5 values at once
  for reroll in range(3): # 3 rerolls
    y[roll_idx] = np.floor(np.random.uniform(0,6,roll_idx.shape)) # roll the dice
    counts = [sum(y == i) for i in range(6)] # count number of each outcome
    max_idx = np.argmax(counts) # which was the most frequent?
    
    if np.max(counts) == 5: # all 5 dice were the same!!!
      print('YAMSLAM!')
      return 1
    
    roll_idx = np.nonzero(y != max_idx)[0]  # update the indices to re-roll
    
  # we've run all 3 re-rolls but still didn't get a yamslam
  return 0      

#### Running multiple trials

In [ ]:
%%time
import numpy as np
yamslam_trials = []
for i in range(100):
  yamslam_trials.append(yamslam())
  
print("Probability of yamslam: {}".format(np.mean(yamslam_trials)))

YAMSLAM!
Probability of yamslam: 0.01
CPU times: user 68.7 ms, sys: 3.94 ms, total: 72.6 ms
Wall time: 76.9 ms


#### Improving the code

```python
counts = [sum(y == i) for i in range(6)]
max_idx = np.argmax(counts)
```

---

- `np.bincounts` produces the same result for `count` as our list comprehension

- More importantly, what is the above code effectively calculating?

In [ ]:
# We're calculating the mode!
from scipy.stats import mode

np.random.seed(0)
y = np.zeros(5)       
roll_idx = np.array(range(5))
y[roll_idx] = np.floor(np.random.uniform(0,6, roll_idx.shape))
roll_idx = np.nonzero(y != mode(y)[0])[0]  

*Tip: before implementing a mathematical operation, check the documentation to see if it's already part of the library -- chances are, it is.*

## **Practice Questions**


Question 1: Dot Product Manually vs Numpy: Create Two random Arrays using Numpy and then calculate their dot product via for loops and then calculate via numpy.dot(). 

We will record the time difference between two. Unfortunately, when we wish to use elapsed times programmatically, cell magics are not quite enough, so we use the ``timeit`` package.


In [ ]:
import timeit

In [ ]:
#Create random vectors a and b or size 10,000 using numpy and then calculate the dot product
# between the two finding the dot product using for loops and then from using numpy.dot. 

## Using for loops

a =   # Initialize a here
b =   # Initialize b here

dot_loops = 0

start_time = timeit.default_timer()

# Write for loop here (update dot_loops)


# End for loop

loop_time = timeit.default_timer() - start_time
print(dot_loops)

In [ ]:
# Using numpy

start_time = timeit.default_timer()
dot_np =        # Call a numpy function here to get the dot product of a and b
np_time = timeit.default_timer() - start_time
print(dot_np)

2535.3354911401298


Submit to autograder

In [ ]:
grader.grade(test_case_id = 'test_case_dot_product', answer = (a, b, dot_loops, loop_time, dot_np, np_time))

Correct! You earned 2.0/2.0 points. You are a star!

Your submission has been successfully recorded in the gradebook.


Question 2: Matrix: Do multiplication on a given (100,100) matrix on a corresponding vector iteratively vs using vectorization

In [ ]:
#Initialize a (100,100) and a (100,1) vector and compute the matrix product of
#the two iteratively and by vectorization.

a =     # Initialize (100,100) matrix here
b =     # Initialize (100,1) vector here


mult_loop = []

start_time = timeit.default_timer()

# Write loop code here


# End loop code here

# Do not modify below here
loop_time = timeit.default_timer() - start_time

mult_loop = np.array(mult_loop)

print(mult_loop)

In [ ]:
# Using numpy

start_time = timeit.default_timer()
mult_np =           # Call a numpy function here to get the matrix-vector product of a and b
np_time = timeit.default_timer() - start_time

print(mult_np)

Submit to autograder

In [ ]:
grader.grade(test_case_id = 'test_case_mult', answer = (a, b, mult_loop, loop_time, mult_np, np_time))

Question 3: Outer Product: Compute the outer product of two vectors (outer product of (100,1) with (1,100) vectors should be (100,100) matrix)

In [ ]:
#Initialize two vectors of size (100, 1) and (1, 100) using numpy (make sure dimensions match)
#Calculate the outer product between them

a =             # Initialize (100, 1) vector here
b =             # Initialize (1, 100) vector here

outer =         # Compute outer product here

print(outer)

Submit to Autograder

In [ ]:
grader.grade(test_case_id = 'test_case_outer', answer = (a, b, outer))

Correct! You earned 2.0/2.0 points. You are a star!

Your submission has been successfully recorded in the gradebook.


Question 4: Elementwise multiplication: Compute the elementwise product of two (1,10000) vectors iteratively and using vectorization

In [ ]:
#Initialize two vectors of size (1, 10000)
#Compute the elementwise product of both iteratively and through vectorization


a =         # Initialize (1, 10000) vector here
b =         # Initialize (1, 10000) vector here


eltwise_loop = []

start_time = timeit.default_timer()

# Loop code here




# End loop code here

# Do not modify below this
loop_time = timeit.default_timer() - start_time
eltwise_loop = np.array(eltwise_loop)

print(eltwise_loop)


[[0.04293528 0.44177341 0.17401767 ... 0.08308864 0.70962016 0.19133979]]


In [ ]:
start_time = timeit.default_timer()
eltwise_np =            # Compute element-wise product of a, b using a np function
np_time = timeit.default_timer() - start_time

print(eltwise_np)

[[0.04293528 0.44177341 0.17401767 ... 0.08308864 0.70962016 0.19133979]]


In [ ]:
grader.grade(test_case_id = 'test_case_eltwise', answer = (a, b, eltwise_loop, loop_time, eltwise_np, np_time))

Correct! You earned 2.0/2.0 points. You are a star!

Your submission has been successfully recorded in the gradebook.


Question 5: Generate a (100,100) matrix where each element is a value of sin(x) on the domain of [0,pi]

In [ ]:
#Generate a (100,100) matrix called data whose values are sin(x) on x -> [0,pi]
# That is, data is a 100 x 100 matrix such that the ij-th element is sin(x) with x 
# sampled uniformly on [0, pi]



In [ ]:
grader.grade(test_case_id = 'test_case_sin', answer = data)

Correct! You earned 2.0/2.0 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## **Summary**

- Before writing a loop, consider if the operation can be *vectorized*
- Vectorization is the application of an operation over an entire array, instead of element by element
- Results in more concise code, and many vectorized implementations of functions are optimized

Super! You have finished this worksheet. If you want to learn other powerful NumPy methods, I strongly encourage you to learn [NumPy Tutorial](https://github.com/ageron/handson-ml/blob/master/tools_numpy.ipynb). And you could practice vectorization by completing array_practice.ipynb.

## **Submitting to the Autograder**

Now go to the File menu and choose "Download .ipynb".  Go to [Gradescope](https://www.gradescope.com/courses/409970) and:

1. From "File" --> Download both .ipynb and .py files
1. Name these files `Vectorization_WS.ipynb` and `Vectorization_WS.py` respectively
1. Sign in using your Penn email address (if you are a SEAS student we recommend using the Google login) and ensure  your class is "CIS 5200"
1. Select **Worksheet: Vectorization**
1. Upload both files
1. PLEASE CHECK THE AUTOGRADER OUTPUT TO ENSURE YOUR SUBMISSION IS PROCESSED CORRECTLY!

You should be set! Note that this assignment has 10 autograded points that will show up upon submission. Points are awarded based on a combination of correctness and sufficient effort. 